In [2]:
import numpy as np
import pandas as pd
import re
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Data preprocessing
data_text = """
Juana. esclava
Domingo veinte y dos de 
y nueve yo Thomas de Orvera baptize, y pusse 
santos oleos a Juana de nacion Mina esclava de
Juan Joseph de Justis fueron sus P.P. Joseph Salcedo
y Ana de Santiago su mugger, y lo firmé.
Lunes veinte y ocho de Diziembre de mil setecientos y veinte y dos as Yo
Thomas de Orvera cura B.do de la Ygla. Parroq.l de San Carlos de
Matanzas Baptize y puse los Ss. Oleos á Thomas negro adulto
n.on Kongo esclavo de Don Juan Joseph de Justis fue su Padre. Joseph n.on
Inglés esclavo del dho Don Juan, y lo firmé.
"""
def text_cleaner(text):
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zñáéíóúü]", " ", newString)
    long_words=[]
    for i in newString.split():
        if len(i)>=3:                  
            long_words.append(i)
    return (" ".join(long_words)).strip()

data_new = text_cleaner(data_text)

# Creating Sequences
def create_seq(text, length = 30):
    sequences = list()
    for i in range(length, len(text)):
        seq = text[i-length:i+1]
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

sequences = create_seq(data_new)

# Character Mapping
chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        encoded_seq = [mapping[char] for char in line]
        sequences.append(encoded_seq)
    return sequences

sequences = encode_seq(sequences)

# Preparing the dataset
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
vocab = len(mapping)
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab)
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Defining the model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer=Adam(learning_rate=0.01))

# Training
callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint('model.h5', save_best_only=True, 
             save_weights_only=False, monitor='val_loss')]

history = model.fit(X_tr, y_tr, epochs=50, batch_size=256,
                    verbose=1, callbacks=callbacks, validation_data=(X_val, y_val))

def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    import heapq
    in_text = seed_text
    predictions = []
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre').squeeze()
        # predict character
        pred = model.predict(np.array([encoded]), verbose=0)
        # applying softmax to convert output into probabilities
        probas = np.exp(pred) / np.sum(np.exp(pred))
        # getting top 3 predictions
        top_3 = heapq.nlargest(3, zip(probas[0], list(range(len(probas[0])))))
        # reverse map integer to character for each prediction and store them
        for score, idx in top_3:
            out_char = ''
            for char, index in mapping.items():
                if index == idx:
                    out_char = char
                    break
            # store the prediction information
            predictions.append({'score': float(score), 'token': idx, 'token_str': out_char, 'sequence': in_text + out_char})
        # continue generating based on the top prediction
        in_text += predictions[-1]['token_str']
    return in_text


print(generate_seq(model, mapping, 30, "Domingo nueve de Junio de mil sete".lower(), 15))


Total Sequences: 387
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 30, 50)            1300      
                                                                 
 gru_1 (GRU)                 (None, 150)               90900     
                                                                 
 dense_1 (Dense)             (None, 26)                3926      
                                                                 
Total params: 96,126
Trainable params: 96,126
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
2/2 [==============================] - 5s 971ms/step - loss: 3.2429 - acc: 0.0891 - val_loss: 3.0668 - val_acc: 0.1282
Epoch 2/50
2/2 [==============================] - 1s 330ms/step - loss: 3.6248 - acc: 0.1609 - val_loss: 3.2186 - val_acc: 0.1026
Epoch 3/50
2/2 [============